<div align="right" style="text-align: right"><i>Peter Norvig<br>April 2015<br>Python 3: Feb 2019<br>Steve's bus: Apr 2020<br>Mad Cheryl: May 2020</i></div>

# When is Cheryl's Birthday?


**[This puzzle](https://en.wikipedia.org/wiki/Cheryl%27s_Birthday)** has been making the rounds:

1. Albert and Bernard became friends with Cheryl, and want to know when her birthday is. Cheryl gives them a list of 10 possible dates:
   - May 15,     May 16,     May 19
   - June 17,    June 18
   - July 14,    July 16
   - August 14,  August 15,  August 17
3. **Cheryl** then privately tells Albert the month and Bernard the day of her birthday.
4. **Albert**: "I don't know when Cheryl's birthday is, and I know that Bernard does not know."
5. **Bernard**: "At first I did't know when Cheryl's birthday is, but I know now."
6. **Albert**: "Then I also know when Cheryl's birthday is."
7. So when is Cheryl's birthday?

Let's work through the puzzle line by line.



## 1. Cheryl gives them a list of 10 possible dates:


In [1]:
DATES = ['May 15',    'May 16',    'May 19',
        'June 17',   'June 18',
        'July 14',   'July 16',
      'August 14', 'August 15', 'August 17']

We'll define accessor functions for the month and day of a date:

In [2]:
def month(date) -> str: return date.split()[0]
def day(date)   -> str: return date.split()[1]

assert month('May 15') == 'May'
assert   day('May 15') == '15'

## 2. Cheryl then privately tells Albert the month and Bernard the day of her birthday.

Now we have to think really carefully about what we're doing. The puzzle is tricky because we're dealing with two types of uncertainty:

1. Albert and Bernard are initially uncertain about the birthdate. *(Cheryl knows something they don't know.)*
2. We (the puzzle solvers) don't know what Albert and Bernard were told. *(They know something we don't know.)*

If Cheryl tells Albert "May", then he believes the birthdate could be either May 15, May 16, or May 19. We'll call `{'May 15', 'May 16', 'May 19'}` his **belief state** about the birthdate. We will say that a person **knows** the birthdate when they get down to a belief state with exactly one possibility. The second type of uncertainty is that we don't know that Albert was told "May", so we don't know his belief state. But we do know some statements about his belief state, and our task is to use those statements to solve the puzzle. 

In [3]:
BeliefState = set # A belief state is a set of possible beliefs; in this problem possible dates of Cheryl's birthday.

The way we will deal with our uncertainty as puzzle solvers is by considering each of the ten dates one at a time and reasoning as follows: 
- If this date were Cheryl's true birthdate, then we would know what Albert and Bernard were told: we would eliminate the second type of uncertainty, and we could figure out their belief states. 
- From that we could figure out if the statements are true (given this date). 
- If the puzzle is correct and we don't make mistakes, then there will be only one date that makes all the statements true; that's Cheryl's birthday.

We can define the idea of Cheryl having **told** someone a component of her birthdate, and  the idea of **knowing** a birthdate as follows:

In [4]:
def told(part: str) -> BeliefState:
    """Cheryl told a part of her birthdate to someone; return a belief state of possible dates."""
    return {date for date in DATES if part in date}

def know(beliefs: BeliefState) -> bool:
    """A person `knows` the answer if their belief state has only one possibility."""
    return len(beliefs) == 1

(Note: one thing I dislike about my code is that the function `told` uses the global `DATES`. Later we will see that the function `cheryls_birthday` does the same. For that to be an acceptable design choice, we need to think of `DATES` as a constant, not a variable.)

Let's see what happens if we consider the date `'May 15'` as a possible birthday.

First, Cheryl tells Albert `'May'` and Bernard `'15'`, giving them these belief states:  

In [5]:
told('May')

{'May 15', 'May 16', 'May 19'}

In [6]:
told('15')

{'August 15', 'May 15'}

We can then check whether the various statements  are true, given `'May 15'` .  The first part of Albert's first statement is  "I don't know when Cheryl's birthday is." We can verify that that part of the statement is true, given that he was told the month:

In [7]:
not know(told('May'))

True

If the rest of the statements work out to be true, then `'May 15'` would be a solution to the puzzle.  If not, it must be one of the other dates.

# Overall Strategy

Here is the main function, `cheryls_birthday`, which computes the subset of dates in the global variable `DATES` that satisfy the statements made by Albert, Bernard, and Albert again.  We will define a **statement** as a boolean function that takes a single date as input and returns true if the statement would be true in the condition that the given date is Cheryl's actual birthday. So a statement only has to consider one date at a time. But the code within a statement may have to consider belief states.

The function `satisfy` takes a belief state (a set of dates) and some statements and returns the subset that satisfies all the statements.

In [8]:
def cheryls_birthday() -> BeliefState:
    """Return a subset of the global `DATES` for which all three statements are true."""
    return satisfy(DATES, albert1, bernard1, albert2)

def satisfy(beliefs: BeliefState, *statements) -> BeliefState:
    """Return the subset of `beliefs` that satisfy all the statements."""
    return {b for b in beliefs if all(statement(b) for statement in statements)}

## 3. Albert: I don't know when Cheryl's birthday is, and I know that Bernard does not know.

The function `albert1` takes as input a single possible birthdate and returns `True` if Albert's statement is true for that birthdate. How do we go from Albert's English statement to a Python function? Let's paraphrase it in a form that uses the concepts we have defined:

**Albert**: *After Cheryl **told** me the **month** of her birthdate, my belief state is a set of dates such that I didn't **know** her birthday.  And I know that Bernard does not know. How do I know that? I can see that for all the possible dates in my belief state, if Bernard was **told** the **day** of that date, he would **not know** Cheryl's birthday.*

That I can translate directly into code:

In [9]:
def albert1(date) -> bool:
    """Albert: I don't know when Cheryl's birthday is, and I know that Bernard does not know."""
    dates = told(month(date))
    return not know(dates) and not satisfy(dates, bernard_knows)

def bernard_knows(date) -> bool: 
    """Does Bernard know the birthday after being told the day?"""
    return know(told(day(date))) 

We haven't solved the puzzle yet, but let's take a peek and see which dates satisfy Albert's statement:

In [10]:
satisfy(DATES, albert1)

{'August 14', 'August 15', 'August 17', 'July 14', 'July 16'}

## 4. Bernard: At first I don't know when Cheryl's birthday is, but I know now.

Again, a paraphrase:

**Bernard:** *At first Cheryl **told** me the **day**, and I didn't **know**.  After I heard Albert's **statement**, I updated my **belief state**, and now I **know**.*

In [11]:
def bernard1(date) -> bool:
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = told(day(date))
    after    = satisfy(at_first, albert1)
    return not know(at_first) and know(after)

Let's see which dates satisfy both Albert and Bernard's statements:

In [12]:
satisfy(DATES, albert1, bernard1)

{'August 15', 'August 17', 'July 16'}

Wait a minute–I thought that Bernard **knew**?! Why are there three possible dates? 

Bernard does indeed know; it is just that we, the puzzle solvers, don't know.  That's because Bernard knows something we don't know: the day. If Bernard was told `'15'` then he would know `'August 15'`; if he was told `'17'` he would know `'August 17'`, and if he was told `'16'` he would know `'July 16'`. *We* don't know because we don't know which of these is the case. We'll need more information (coming in statement `albert2`) before *we* know.



## 5. Albert: Then I also know when Cheryl's birthday is.

A paraphrase:

**Albert**: *After being told the month and heaaring Bernard's statement, I now know Cheryl's birthday/*

In [13]:
def albert2(date) -> bool:
    "Albert: Then I also know when Cheryl's birthday is." 
    now = satisfy(told(month(date)), bernard1)
    return know(now)

## 6. So when is Cheryl's birthday?



In [14]:
cheryls_birthday()

{'July 16'}

**Success!** We have deduced that Cheryl's birthday is **July 16**. It is now `True` that we know Cheryl's birthday:

In [15]:
know(cheryls_birthday())

True

___

# New Puzzle: Steve's Bus

Here's [another puzzle](https://www.reddit.com/r/riddles/comments/fw7h42/a_riddle_i_couldnt_solve/) that seems to have a very similar format:

1. Steve tells Alice the hour of his bus departure and he tells Annie at which minute it leaves. He also tells them both that the bus leaves between 06:00 and 10:00.
2. Alice and Annie consult the timetable and find the following services between those two time:
   - 06:32, 06:43, 06:50, 07:17, 07:46, 08:19, 08:32, 09:17, 09:19, 09:50.
4. Alice then says “I don’t know when Steve’s bus leaves but I am sure that neither does Annie”
5. Annie Replies “I didn’t know his bus, but now I do”
6. Alice responds “Now I do as well!”
7. When is Steve’s bus?

Upon closer inspection, not only is it a similar format, it is **exactly** the same puzzle, except that months are changed to hours and days to minutes.  If we rewrite the times in the same format as `DATES`, we can solve the problem without writing a single line of new code:

In [16]:
times = {'06:32', '06:43', '06:50', '07:17', '07:46', '08:19', '08:32', '09:17', '09:19', '09:50'}
DATES = {time.replace(':', ' ') for time in times}

cheryls_birthday()

{'08 32'}

Steve took the 8:32 bus.

# Another New Puzzle: Evil Mad Scientist Cheryl

![](https://norvig.com/images/cheryl-trolley.png)

Again, we can solve this problem just by changing the format of `DATES`:

In [17]:
pads = {'A2', 'A3', 'A6', 'B4', 'B5', 'C1', 'C3', 'D1', 'D2', 'D4'}
DATES = {' '.join(pad) for pad in pads}

cheryls_birthday()

{'C 3'}

Mad scientist Cheryl was refering to pad C3. (But may I point out that this Cheryl is not actually a mad scientist, just a [mad engineer](https://www.evilmadscientist.com/2015/evil-mad-engineers/). A true mad scientist would kill 25 people and use the other 25 as a control group.)